#  Loading required python packages

In [ ]:
import pandas as pd    #dataframe and analysis
import geopy.distance  #distance of coordinates
import numpy as np     #for cleaning and maths
import glob            #for specific pattern recognition

# Collecting all files
    
   #### Here we are working with data from a cyclistic 12 months dataset. all are in same type (.csv) files.

In [ ]:
all_files=glob.glob(r'C:\Users\mahad\Downloads\capstone_project\project_1\*.csv')

In [ ]:
all_files

#  Concatenating all files into one

In [ ]:
yearly= pd.concat((pd.read_csv(file)
                  for file in all_files),ignore_index=True)

## inspecting files for inconsistancy, null value and data typtes.

In [ ]:
yearly.describe()

In [14]:
yearly.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3489748 entries, 0 to 3489747
Data columns (total 13 columns):
 #   Column              Non-Null Count    Dtype  
---  ------              --------------    -----  
 0   ride_id             3489748 non-null  object 
 1   rideable_type       3489748 non-null  object 
 2   started_at          3489748 non-null  object 
 3   ended_at            3489748 non-null  object 
 4   start_station_name  3367573 non-null  object 
 5   start_station_id    3366947 non-null  object 
 6   end_station_name    3346506 non-null  object 
 7   end_station_id      3346045 non-null  object 
 8   start_lat           3489748 non-null  float64
 9   start_lng           3489748 non-null  float64
 10  end_lat             3485010 non-null  float64
 11  end_lng             3485010 non-null  float64
 12  member_casual       3489748 non-null  object 
dtypes: float64(4), object(9)
memory usage: 346.1+ MB


In [15]:
yearly.columns

Index(['ride_id', 'rideable_type', 'started_at', 'ended_at',
       'start_station_name', 'start_station_id', 'end_station_name',
       'end_station_id', 'start_lat', 'start_lng', 'end_lat', 'end_lng',
       'member_casual'],
      dtype='object')

# convert date and time *datetime* readable

In [16]:
yearly['started_at'] =  pd.to_datetime(yearly['started_at'], format='%Y-%m-%d %H:%M:%S')


In [17]:
yearly['ended_at'] =  pd.to_datetime(yearly['ended_at'], format='%Y-%m-%d %H:%M:%S')

In [18]:
yearly.dtypes

ride_id                       object
rideable_type                 object
started_at            datetime64[ns]
ended_at              datetime64[ns]
start_station_name            object
start_station_id              object
end_station_name              object
end_station_id                object
start_lat                    float64
start_lng                    float64
end_lat                      float64
end_lng                      float64
member_casual                 object
dtype: object

# Calculate the riding time


In [19]:
yearly['riding_time'] = (yearly['ended_at'] - yearly['started_at'])/pd.Timedelta(minutes=1)

checking the result. we find some unusual vaules so we inspect them. there are some negetive values and some illogically big vaules which caused by corrupted data we need to fillter them. we took maximum value of 24 hour.

In [23]:
print (yearly['riding_time'].max())
print (yearly['riding_time'].min())

58720.03333333333
-29049.966666666667


In [21]:
yearly.describe()

,start_lat,start_lng,end_lat,end_lng,riding_time
count,3.489748e+06,3.489748e+06,3.485010e+06,3.485010e+06,3.489748e+06
mean,4.190417e+01,-8.764494e+01,4.190444e+01,-8.764522e+01,2.476664e+01
std,4.364222e-02,2.575969e-02,4.373705e-02,2.589123e-02,3.904216e+02
min,4.164000e+01,-8.787000e+01,4.154000e+01,-8.807000e+01,-2.904997e+04
25%,4.188224e+01,-8.765888e+01,4.188266e+01,-8.765917e+01,7.883333e+00
50%,4.190000e+01,-8.764170e+01,4.190068e+01,-8.764275e+01,1.451667e+01
75%,4.193000e+01,-8.762773e+01,4.193120e+01,-8.762775e+01,2.663333e+01
max,4.208000e+01,-8.752000e+01,4.216000e+01,-8.744000e+01,5.872003e+04


In [24]:
yearly = yearly[yearly['riding_time'].between(0,1440)]

In [25]:
print (yearly['riding_time'].max())
print (yearly['riding_time'].min())

1439.9
0.0


# Creating weekday for analysis

In [27]:
yearly['day_of_week'] = yearly['started_at'].dt.day_name()

C:\Users\mahad\AppData\Local\Temp/ipykernel_1484/3405633930.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  yearly['day_of_week'] = yearly['started_at'].dt.day_name()


In [28]:
yearly['day_of_week']

0             Sunday
1             Friday
2          Wednesday
3            Tuesday
4           Saturday
             ...    
3489743     Saturday
3489744      Tuesday
3489745      Tuesday
3489746       Monday
3489747       Friday
Name: day_of_week, Length: 3476314, dtype: object

In [29]:
yearly['day_of_week'].unique()

array(['Sunday', 'Friday', 'Wednesday', 'Tuesday', 'Saturday', 'Thursday',
       'Monday'], dtype=object)

# classifying riding hour 

In [31]:
yearly['riding_hour'] = yearly['started_at'].dt.hour

C:\Users\mahad\AppData\Local\Temp/ipykernel_1484/1411790220.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  yearly['riding_hour'] = yearly['started_at'].dt.hour


In [32]:
yearly['riding_hour']

0          17
1          17
2          17
3          12
4          10
           ..
3489743    14
3489744    11
3489745    11
3489746    18
3489747    17
Name: riding_hour, Length: 3476314, dtype: int64

In [33]:
yearly['riding_hour'].unique()

array([17, 12, 10, 14, 15, 18, 13,  2, 16,  8, 20, 19, 11, 23,  6,  9,  7,
       22, 21,  1,  5,  0,  4,  3], dtype=int64)

In [35]:
conditions = [
    (yearly['riding_hour'].between(0,5)),
    (yearly['riding_hour'].between(5,8)),
    (yearly['riding_hour'].between(8,12)),
    (yearly['riding_hour'].between(12,15)),
    (yearly['riding_hour'].between(15,18)),
    (yearly['riding_hour'].between(18,24))
]

values = ['night_ride','early_morning_ride','morning_ride', 'noon_ride', 'afternoon_ride', 'evening_ride']

yearly['riding_hour'] = np.select(conditions, values)

C:\Users\mahad\AppData\Local\Temp/ipykernel_1484/3007892369.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  yearly['riding_hour'] = np.select(conditions, values)


In [36]:
yearly['riding_hour']

0          afternoon_ride
1          afternoon_ride
2          afternoon_ride
3            morning_ride
4            morning_ride
                ...      
3489743         noon_ride
3489744      morning_ride
3489745      morning_ride
3489746    afternoon_ride
3489747    afternoon_ride
Name: riding_hour, Length: 3476314, dtype: object

In [37]:
yearly['riding_hour'].unique()

array(['afternoon_ride', 'morning_ride', 'noon_ride', 'night_ride',
       'early_morning_ride', 'evening_ride'], dtype=object)